In [1]:
import re
import csv
import sqlite3
import pandas as pd
from pathlib import Path

/var/folders/nj/wd0p0vs57mz6bm0lqmcpm4th0000gn/T/ipykernel_48170/2925816044.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Load the CSV file
csv_file = 'covid_submissions_all_info.csv'
data = pd.read_csv(csv_file)

/var/folders/nj/wd0p0vs57mz6bm0lqmcpm4th0000gn/T/ipykernel_48170/3371146526.py:3: DtypeWarning: Columns (3,6,7,9,10,11,12,13,14,15,18,19,27,28,29,30,31,32,33,34,35,36,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,66,67,69,70,71,72,73,74,75,76,84,85,86,87,89,90,91,92,93,94,95,96,104,105,106,107,109,110,111,112,113,114,115,116,124,125,126,127,129,130,131,132,133,134,135,136,144,145,146,147,149,150,151,152,153,154,155,156,164,165,166,167,169,170,171,172,173,174,175,176,184,185,186,187,189,190,191,192,193,194,195,196,204,205,206,207,209,210,211,212,213,214,215,216,224,225,226,227,229,230,231,232,233,234,235,236,244,245,246,247,249,253,254,255,256,257,258,259,275,276) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(csv_file)


In [3]:
# Define a function to count the number of 'rule of five' violations
def rule_of_five_violations(row):
    violations = sum([
        row['HBD'] > 5,
        row['HBA'] > 10,
        row['MW'] >= 500,
        row['cLogP'] >= 5
    ])
    return violations

# Apply the function to the data
data['rule_of_five_violations'] = data.apply(rule_of_five_violations, axis=1)

# Filter out compounds that violate more than one rule
rule_of_five_data = data[data['rule_of_five_violations'] <= 1]

# Extract data for the 'compounds' and 'rule_of_five_compliant_compounds' tables
smiles_data = data[['CID', 'SMILES', 'HBD', 'HBA', 'MW', 'cLogP']].drop_duplicates()

# Select relevant columns for 'assays'
columns_of_interest = ['CID', 'f_avg_IC50', 'r_avg_IC50']
assay_data = data[columns_of_interest]

# Create a new SQLite database
database_file = 'assays_database.db'
conn = sqlite3.connect(database_file)
cursor = conn.cursor()

# Create the 'compounds' table 
cursor.execute('''
    CREATE TABLE IF NOT EXISTS compounds (
        CID TEXT PRIMARY KEY,
        SMILES TEXT,
        HBD INTEGER,
        HBA INTEGER,
        MW REAL,
        cLogP REAL
    );           
''')

# Create the 'rule_of_five_compliant_compounds' table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS rule_of_five_compliant_compounds (
        CID TEXT PRIMARY KEY,
        SMILES TEXT,
        HBD INTEGER,
        HBA INTEGER,
        MW REAL,
        cLogP REAL
    );
''')

# Populate the 'compounds' table
for _, row in smiles_data.iterrows():
    cursor.execute('''
        INSERT OR IGNORE INTO compounds (CID, SMILES, HBD, HBA, MW, cLogP) 
        VALUES (?, ?, ?, ?, ?, ?)
    ''', (row['CID'], row['SMILES'], row['HBD'], row['HBA'], row['MW'], row['cLogP']))

# Populate the 'rule_of_five_compliant_compounds' table
for _, row in rule_of_five_data.iterrows():
    cursor.execute('''
        INSERT OR IGNORE INTO rule_of_five_compliant_compounds (CID, SMILES, HBD, HBA, MW, cLogP) 
        VALUES (?, ?, ?, ?, ?, ?)
    ''', (row['CID'], row['SMILES'], row['HBD'], row['HBA'], row['MW'], row['cLogP']))

# Create the 'assays' table with a foreign key
cursor.execute('''
    CREATE TABLE IF NOT EXISTS assays (
        CID TEXT,
        f_avg_IC50 REAL,
        r_avg_IC50 REAL,
        FOREIGN KEY (CID) REFERENCES compounds(CID)
    );
''')

# Populate the 'assays' data into the table
for _, row in assay_data.iterrows():
    cursor.execute('''
        INSERT INTO assays (CID, f_avg_IC50, r_avg_IC50) 
        VALUES (?, ?, ?)
    ''', (row['CID'], row['f_avg_IC50'], row['r_avg_IC50']))

# Commit changes and close the connection
conn.commit()

# Demonstrate an SQL query for joining the two tables
cursor.execute('''
    SELECT c.CID, c.SMILES, a.f_avg_IC50, a.r_avg_IC50
    FROM compounds c
    JOIN assays a ON c.CID = a.CID
''')

# Fetch and print the result of the query
joined_data = cursor.fetchall()
for row in joined_data:
    print(row)

conn.close()

print("Database and tables created successfully with the data.")

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

